Changing cwd to Root Directory

In [1]:
import os
os.chdir('../')
%pwd

'e:\\code\\project\\Text-Summarizer'

Creating Entity

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)              #Its a configuraton class for data transformation which will be used by other classes
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

Creating Configuration Manager

In [3]:
from textSummarizer.utils import read_yaml,create_directories 
from textSummarizer import constants

class ConfigurationManager:      #class to read the config file and params file
    def __init__(
            self,
            config_file_path = constants.CONFIG_FILE_PATH,       #constants is a file which contains all the paths 
            params_file_path = constants.PARAMS_FILE_PATH
            ):
         
        self.config = read_yaml(config_file_path)      #read_yaml is a function which reads the yaml file and returns a dictionary
        self.params = read_yaml(params_file_path)

        #config is a ConfigBox(shown in trials.ipynb) object which is a dictionary with dot notation access with help of read_yaml function
        create_directories([self.config.artifacts_root])              #create_directories is a function which creates directories if they don't exist; artifacts_root is the path where all the artifacts will be stored present in the config.yaml file


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])      #root_path is the path where all the data will be stored present in the config.yaml file

        return DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name)
        

Creating Components

In [4]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

class DataTransformation:       #class to transform the data
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)      #AutoTokenizer is a class from transformers library which is used to tokenize the data

    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):           #start data transformation
        logger.info("Starting data transformation")

        logger.info("Loading dataset")
        dataset_samsum = load_from_disk(self.config.data_path)       #load_from_disk is a function from datasets library which loads the dataset from the path provided
        
        logger.info("Converting dataset to features")
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features,batched=True)      #map is a function from datasets library which applies the function to the dataset

        logger.info("Saving dataset")        
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,'samsum_dataset'))        #save_to_disk is a function from datasets library which saves the dataset to the path provided
        
        logger.info("Data transformation completed")


d:\anaconda\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Creating Pipeline

In [5]:
try:
    config = ConfigurationManager().get_data_transformation_config()
    config = DataTransformation(config=config)
    config.convert()

except Exception as e:
    logger.info(f"error in data transformation: {e}")
    raise e

d:\anaconda\envs\textS\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HeliosXx\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
d:\anaconda\envs\textS\lib\site-packages\dill\_dill.py:412: PicklingWarning: Cannot locate reference to <class '__main__.DataTransformation'>.
  Sto